Consulta 4
Esta funcion devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Importamos las librerías necesarias para el procesamiento de los datos

In [10]:
import pandas as pd
import re

In [11]:
df_games = pd.read_csv(r'C:\Users\EFRAIN LINARES\Desktop\Henry PI 1\ETL\games_clean.csv')
df_reviews = pd.read_parquet(r'C:\Users\EFRAIN LINARES\Desktop\Henry PI 1\PI MLOps - STEAM\PARQUET\df_review_clean.parquet')
df_sentimientos = pd.read_parquet(r'C:\Users\EFRAIN LINARES\Desktop\Henry PI 1\PI MLOps - STEAM\PARQUET\analisis_sentiment.parquet')

df_games_copy = df_games.copy()
df_reviews_copy = df_reviews.copy()
df_sentimientos_copy = df_sentimientos.copy()

In [12]:
df_games_copy

,genres,app_name,release_date,tags,price,id,developer
0,Action,Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,761140.0,Kotoshiro
1,Indie,Ironbound,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",NaN,643980.0,Secret Level SRL
2,Casual,Real Pool 3D - Poolians,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",Free,670290.0,Poolians.com
3,Action,弹炸人2222,2017-12-07,"['Action', 'Adventure', 'Casual']",0.99,767400.0,彼岸领域
4,Action,Log Challenge,NaN,"['Action', 'Indie', 'Casual', 'Sports']",2.99,773570.0,NaN
...,...,...,...,...,...,...,...
32130,Casual,Colony On Mars,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",1.99,773640.0,"Nikita ""Ghost_RUS"""
32131,Casual,LOGistICAL: South Africa,2018-01-04,"['Strategy', 'Indie', 'Casual']",4.99,733530.0,Sacada
32132,Indie,Russian Roads,2018-01-04,"['Indie', 'Simulation', 'Racing']",1.99,610660.0,Laush Dmitriy Sergeevich
32133,Casual,EXIT 2 - Directions,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",4.99,658870.0,"xropi,stev3ns"


Combinamos df1 y df2 en base al ID del ítem utilizando un join izquierdo (manteniendo todas las filas de df1) y luego impremimos el DataFrame resultante.

In [13]:
# Combinar df1 y df2 en base al ID del ítem utilizando un left join (manteniendo todas las filas de df1)
df_merged = df_sentimientos_copy.merge(df_games_copy[['id', 'developer']], left_on='item_id', right_on='id', how='left')

# Imprimir el DataFrame resultante
df_merged

ValueError: You are trying to merge on object and float64 columns for key 'item_id'. If you wish to proceed you should use pd.concat

In [14]:
# Define un patrón regex para extraer el año (cuatro dígitos)
patron = r'(\b\d{4}\b)'

# Extrae el año de cada celda en la columna 'posted' usando regex
df_merged['year_posted'] = df_merged['posted'].str.extract(patron)

# Elimina la columna original 'posted'
df_merged.drop(columns=['posted'], inplace=True)

NameError: name 'df_merged' is not defined

In [16]:
nuevo_orden_columnas = [
    'user_id',
    'item_id',
    'recommend',
    'sentiment_analysis',
    'developer',
    'year_posted'
]

# Reordena las columnas del DataFrame con el nuevo orden
df_merged = df_merged[nuevo_orden_columnas]

df_merged


NameError: name 'df_merged' is not defined

In [17]:
# Eliminar filas con valores NaN en la columna 'year_posted'
df_merged.dropna(subset=['year_posted'], inplace=True)

# Convertir la columna 'year_posted' a tipo de datos entero
df_merged['year_posted'] = df_merged['year_posted'].astype(int)

NameError: name 'df_merged' is not defined

Creamos un diccionario que contiene los tres principales desarrolladores con la mayoría de las recomendaciones positivas para un año dado.

Filtramos los datos para el año específico, luego selecciona los juegos con recomendaciones verdaderas y el análisis de sentimientos más alto.
Agrupamos los resultados por desarrollador, contando las recomendaciones, los ordenamos en orden descendente y seleccionamos los tres principales.
Creamos un diccionario con los tres principales desarrolladores y sus puestos correspondientes.

In [18]:
def best_developer_year(df, año):
    
    # Filtrar el DataFrame para el año dado
    df_año = df[df['year_posted'] == año]

    # Filtrar por recomendaciones verdaderas y sentimiento de análisis más alto
    df_filtrado = df_año[df_año['recommend'] & (df_año['sentiment_analysis'] == df_año['sentiment_analysis'].max())]

    # Agrupar por desarrollador y contar las recomendaciones
    top_developers = df_filtrado.groupby('developer')['recommend'].sum()

    # Ordenar los desarrolladores por número de recomendaciones en orden descendente
    top_developers = top_developers.sort_values(ascending=False)

    # Tomar los top 3 desarrolladores
    top_3_developers = top_developers.head(3)

    # Crear el diccionario ordenado con los top 3 desarrolladores
    top_developers_dict = {}
    for i, (developer, recomendaciones) in enumerate(top_3_developers.items(), 1):
        puesto = f"Puesto {i}"
        top_developers_dict[puesto] = developer

    return top_developers_dict

In [19]:
# Solicitar al usuario que ingrese el año deseado
año = int(input("Steam solo tiene datos de reseñas de usuarios para los años 2010-2015. Ingrese un año entre 2010 y 2015, inclusive: "))

# Check if the input year is within the desired range (2010-2015)
if año < 2010 or año > 2015:
    print("Error: El año ingresado debe estar entre 2010 y 2015, inclusive.")
else:
    # Call the function with the desired year and DataFrame df_merged
    top_3_desarrolladores = best_developer_year(df_merged, año)
    
    # Imprimir el top 3 de desarrolladores para el año dado
    print(f"Top 3 de desarrolladores para el año {año}: {top_3_desarrolladores}")

ValueError: invalid literal for int() with base 10: 'aklfjsmgfs'